In [ ]:
import cv2
import torch
import numpy as np
import time

In [ ]:
cap=cv2.VideoCapture(0)
time.sleep(2)

In [ ]:
for i in range(30):
    ret,bg=cap.read()
print('initial bg done')

initial bg done


In [ ]:
bg_tensor = torch.from_numpy(bg).float() / 255.0

In [ ]:
lower_red1=torch.tensor([0,120,70],dtype=torch.float32)/255.0
upper_red1=torch.tensor([10,255,255],dtype=torch.float32)/255.0

lower_red2=torch.tensor([170,120,70],dtype=torch.float32)/255.0
upper_red2=torch.tensor([180,255,255],dtype=torch.float32)/255.0
'''lower_pink=torch.tensor([140,50,150],dtype=torch.float32)/255.0
upper_pink=torch.tensor([170,255,255],dtype=torch.float32)/255.0'''

In [ ]:
while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        print("error")
        break
    frame=cv2.flip(frame,1)
    frame_tensor=torch.from_numpy(frame).float()/255.0

    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    hsv_tensor=torch.from_numpy(hsv).float()/255.0

    mask1=(hsv_tensor>=lower_red1)&(hsv_tensor<=upper_red1)
    mask2=(hsv_tensor>=lower_red2)&(hsv_tensor<=upper_red2)
    mask=mask1|mask2

    #ask=(hsv_tensor>=lower_pink)&(hsv_tensor<=upper_pink)

    cloak=bg_tensor*mask

    remaining_area=frame_tensor*(~mask)

    final=remaining_area+cloak

    output=(final.numpy()*255).astype(np.uint8)

    cv2.imshow('invisibility cloak',output)
    if cv2.waitKey(1)==27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import time

# Initialize the camera
cap = cv2.VideoCapture(0)
time.sleep(2)  # Let the camera warm up

# Capture the background frame for 2 seconds
for i in range(30):
    ret, background = cap.read()

# Define the color range for the cloak (for example: red cloak)
lower_red = np.array([0, 120, 70])
upper_red = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for consistency
    frame = cv2.flip(frame, 1)

    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask for the cloak color
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2

    # Refine the mask using morphological operations
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))

    # Create an inverse mask to get the remaining part of the frame
    mask_inv = cv2.bitwise_not(mask)

    # Extract the background where the cloak is detected
    cloak_area = cv2.bitwise_and(background, background, mask=mask)

    # Extract the remaining part of the frame
    remaining_area = cv2.bitwise_and(frame, frame, mask=mask_inv)

    # Combine both areas
    final_output = cv2.addWeighted(cloak_area, 1, remaining_area, 1, 0)

    # Display the result
    cv2.imshow('Invisibility Cloak', final_output)

    if cv2.waitKey(1) == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
